In [ ]:
from Bio import Entrez, SeqIO
from pathlib import Path
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def buscar_genoma(nome_virus):
    Entrez.email = "computandocombiopy@gmail.com"
    termo_de_busca = f"{nome_virus} complete genome"
    busca = Entrez.esearch (db = "nucleotide", term = termo_de_busca, retmax = 1)
    resultado = Entrez.read(busca)
    busca.close()
    
    id_do_virus = (resultado["IdList"])

    if not id_do_virus:
        print(f"Não há um vírus {nome_virus} no NCBI. Tente outra vez!")

    return id_do_virus[0]


def baixar_genoma(iden, output_folder=Path("data")/"genomes"):
    #Faz os genomas irem pra uma pasta relativa ao repositório
    output_folder.mkdir(parents=True, exist_ok=True)
    filename= output_folder/f"{iden}.fasta"
    
    #Acessando o Entrez
    Entrez.email="computandocombiopy@gmail.com"

    #Checo se o arquivo fasta existe, se não ele o cria
    if not filename.exists():
        #Baixa o arquivo .fasta (funciona para GenBank e RefSeq)
        stream= Entrez.efetch(
            db="nucleotide",id=iden, rettype="fasta", retmode="text"
        )
        data = stream.read()
        #Checa se data tá vazio ou não
        if not data:
            print("Error: Resposta vazia do NCBI.")
        else:
            with open(filename, "w") as output:
                output.write(data)
        print(f"Salvo em {filename}")

def rodar_muscle(entrada_fasta, saida_fasta): #recebe 2 arquivos como parâmetros, um genomas.fasta e outro genomas_alinhados.fasta (onde o MUSCLE salva o resultado do alinhamento múltiplo)
    muscle_exe = "muscle-win64.v5.3.exe"  # acessa o executável do MUSCLE
    comando = [muscle_exe, "-align", entrada_fasta, "-output", saida_fasta] #alinha o arquivo de entrada e coloca os dados de saída no arquivo criado pelo executável, no caso genomas_alinhados.fasta, que foi passado como parâmetro
    
    #tratamento de erro
    try: 
        resultado = subprocess.run(comando, check=True, capture_output=True, text=True) 
        print("MUSCLE finalizado com sucesso.")
        print("STDOUT:", resultado.stdout)
        print("STDERR:", resultado.stderr)
    except subprocess.CalledProcessError as e:
        print("Erro ao rodar MUSCLE:")
        print("STDOUT:", e.stdout)
        print("STDERR:", e.stderr)
        raise #imprime o erro novamente, além de interromper o programa, impedindo que o erro se prolongue.

def plotar_matriz_triangular(nomes, matriz): #recebe uma lista de nomes e uma matriz de similaridade, com valores entre 0 e 1
    matriz_np = np.array(matriz) #garantindo que a matriz é um array NumPy, necessário para facilitar as operações matemáticas e criação de gráficos

    # Máscara para esconder a parte superior da matriz (acima da diagonal)
    mask = np.triu(np.ones_like(matriz_np, dtype=bool))

    plt.figure(figsize=(8, 6)) #criando figura com tamanho definido em polegadas (largura, altura)
    sns.heatmap(matriz_np, annot=True, fmt=".2f", mask=mask,
                xticklabels=nomes, yticklabels=nomes,
                cmap="YlGnBu", linewidths=0.5, cbar_kws={"label": "Similaridade"},
                vmin=0.001, vmax=1.0)

    plt.title("Matriz de Similaridade Genômica", fontsize=16)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()